In [ ]:
from transformers import AutoModelForSequenceClassification, DistilBertTokenizer
from datasets import Dataset
from sklearn.metrics import accuracy_score

import torch
import pandas as pd

In [ ]:
num_labels = 3
model_ckp = 'distilbert-base-uncased'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
# Load the data
data = Dataset.from_csv('./synthetic_data.csv')
cats = {cat: i for i, cat in enumerate(set(data['category']))}
labels = [cats[i] for i in data['category']]
data = data.add_column(name='labels', column=labels)
data = data.shuffle()
data = data.train_test_split(test_size=0.1)
data

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained(model_ckp)
model = (AutoModelForSequenceClassification.from_pretrained(model_ckp, num_labels=num_labels)).to(device)

def tokenize(batch, tokenizer):
    return tokenizer(batch["entry"], padding=True, truncation=True)

encoded = data.map(tokenize, batched=True, batch_size=None, fn_kwargs={'tokenizer': tokenizer})

In [ ]:
model

In [ ]:
type(model)